In [1]:
from tensorflow import keras

## CNN 모델을 이용한 가위-바위-보 분류 데이터 셋 분류

### Step 1. Input tensor 와 Target tensor 준비(훈련데이터)

(1) 가위-바위-보 데이터셋 다운로드

In [2]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps.zip'

In [3]:
path_zip = keras.utils.get_file(fname='rps.zip', origin=url,
                                extract=True, cache_dir='/content')

In [4]:
url = 'https://storage.googleapis.com/laurencemoroney-blog.appspot.com/rps-test-set.zip'

In [5]:
path_zip = keras.utils.get_file(fname='rps-test-set.zip', origin=url,
                                extract=True, cache_dir='/content')

(2) ImageDataGenerator를 이용해 이미지 파일을 load 하기 위한 경로 지정

In [6]:
train_path = '/content/datasets/rps'

In [7]:
test_path = '/content/datasets/rps-test-set'

(3) ImageDataGenerator 객체 생성  
* 객체 생성 시 rescale 인자를 이용하여 텐서 내 원소의 범위를 [0 ~ 255] => [0 ~ 1] 로 ReScaling 진행

In [8]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [9]:
train_datagen = ImageDataGenerator(rescale=1./255., validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255.)

* .flow_from_directory() 메서드를 이용하여 학습데이터와 검증데이터를 위한 DirectoryIterator 객체 생성

In [10]:
# class_mode = 'categorical' -> one_hot encoding
# training -> validation_split=0.2에 의해 80%만 가져옴 

train_generator = train_datagen.flow_from_directory(directory=train_path, target_size=(150,150), batch_size=20, shuffle=True, class_mode='categorical', subset = 'training')

validation_generator = train_datagen.flow_from_directory(directory=train_path, target_size=(150,150), batch_size=20, shuffle=True, class_mode='categorical', subset='validation')

test_generator = test_datagen.flow_from_directory(directory=test_path, target_size=(150,150), batch_size=20, shuffle=False, class_mode='categorical')

Found 2016 images belonging to 3 classes.
Found 504 images belonging to 3 classes.
Found 372 images belonging to 3 classes.


### Step2. CNN 모델 디자인 및 학습 정보 설정


(1) 모델을 생성하여 return 할 수 있는 build_model 함수 정의
  * CNN 모델 디자인
  * CNN 모델의 summary 정보 출력
  * 모델의 학습 정보 설정

In [11]:
from tensorflow.keras import models, layers

In [12]:
model = models.Sequential()

In [13]:
# 1st conv
#  -filter : 32 - kernel : (3, 3), activation : relu

model.add(layers.Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(150,150,3)))

# 1st max pooing
#  - pooing_size : (2,2)

model.add(layers.MaxPooling2D(pool_size=(2,2)))

# 2nd conv
#  -filter : 64 - kernel : (3, 3), activation : relu

model.add(layers.Conv2D(filters=64, kernel_size=(3,3), activation='relu'))

# 2nd max pooing
#  - pooing_size : (2,2)

model.add(layers.MaxPooling2D(pool_size=(2,2)))

# 3rd conv
#  -filter : 128 - kernel : (3, 3), activation : relu

model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))

# 3rd max pooing
#  - pooing_size : (2,2)

model.add(layers.MaxPooling2D(pool_size=(2,2)))

# 4th conv
#  -filter : 128 - kernel : (3, 3), activation : relu

model.add(layers.Conv2D(filters=128, kernel_size=(3,3), activation='relu'))

# 4th max pooing
#  - pooing_size : (2,2)

model.add(layers.MaxPooling2D(pool_size=(2,2)))

# flatten

model.add(layers.Flatten())

# classification_module
model.add(layers.Dense(units=512, activation='relu'))
model.add(layers.Dense(units=3, activation='softmax'))

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [15]:
# one-hot = categorical / 분류 = crossentropy
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Step 3. 모델에 데이터 generator 연결 후 학습 
  * model.fit() 이용하여 데이터 연결 및 학습
  * 학습 과정은 history 변수에 저장

In [16]:
model.fit(train_generator, epochs=10, steps_per_epoch=len(train_generator), validation_data=validation_generator, validation_steps=len(validation_generator))

Epoch 1/10
101/101 [==============================] - 12s 93ms/step - loss: 0.7456 - accuracy: 0.7822 - val_loss: 0.6516 - val_accuracy: 0.8274
Epoch 2/10
101/101 [==============================] - 9s 90ms/step - loss: 0.0784 - accuracy: 0.9802 - val_loss: 0.8031 - val_accuracy: 0.8214
Epoch 3/10
101/101 [==============================] - 9s 90ms/step - loss: 1.1415e-04 - accuracy: 1.0000 - val_loss: 1.1781 - val_accuracy: 0.8373
Epoch 4/10
101/101 [==============================] - 9s 86ms/step - loss: 0.2188 - accuracy: 0.9906 - val_loss: 0.5796 - val_accuracy: 0.8611
Epoch 5/10
101/101 [==============================] - 9s 84ms/step - loss: 5.3515e-06 - accuracy: 1.0000 - val_loss: 0.8263 - val_accuracy: 0.8790
Epoch 6/10
101/101 [==============================] - 8s 84ms/step - loss: 0.1450 - accuracy: 0.9931 - val_loss: 1.3230 - val_accuracy: 0.7599
Epoch 7/10
101/101 [==============================] - 8s 83ms/step - loss: 1.7403e-06 - accuracy: 1.0000 - val_loss: 1.4914 - val_acc

### Step 4. 테스트 데이터 셋을 통한 모델의 성능 평가

In [17]:
loss, accuracy = model.evaluate(test_generator)

19/19 [==============================] - 1s 74ms/step - loss: 3.8790 - accuracy: 0.7769
